In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [249]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-20 13:27:09--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-07-20 13:27:09 (22.8 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [250]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [251]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [252]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [253]:
!pip install nltk

In [254]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [255]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [256]:
# tokenize
tokens = word_tokenize(text.lower())

In [257]:
vocab = {'<unk>':0}
for token in Counter(tokens).keys():
    if token not in vocab:
        vocab[token] = len(vocab)

In [258]:
len(vocab)

12343

In [259]:
input_sentences = text.split('\n')

In [260]:
len(input_sentences)

40001

In [261]:
input_sentences = [s for s in input_sentences if s != ""]

In [262]:
merged_sentences = []

# pick how many lines to merge — you said 4–5, so we’ll do 5
chunk_size = 5

# make sure your original list has no empty lines
input_sentences = [s.strip() for s in input_sentences if s.strip()]

# loop in steps of `chunk_size`
for i in range(0, len(input_sentences), chunk_size):
    chunk = input_sentences[i:i+chunk_size]
    merged = ' '.join(chunk)
    merged_sentences.append(merged)

print(merged_sentences[:3])  # preview first 3 chunks

['First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen:', 'You are all resolved rather to die than to famish? All: Resolved. resolved. First Citizen: First, you know Caius Marcius is chief enemy to the people.', "All: We know't, we know't. First Citizen: Let us kill him, and we'll have corn at our own price. Is't a verdict?"]


In [263]:
len(merged_sentences)

6556

In [264]:
merged_sentences[:5]

['First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen:',
 'You are all resolved rather to die than to famish? All: Resolved. resolved. First Citizen: First, you know Caius Marcius is chief enemy to the people.',
 "All: We know't, we know't. First Citizen: Let us kill him, and we'll have corn at our own price. Is't a verdict?",
 "All: No more talking on't; let it be done: away, away! Second Citizen: One word, good citizens. First Citizen:",
 'We are accounted poor citizens, the patricians good. What authority surfeits on would relieve us: if they would yield us but the superfluity, while it were wholesome, we might guess they relieved us humanely; but they think we are too dear: the leanness that']

In [265]:
def text_to_indices(sentence, vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [266]:
input_numerical_sentences = []

for sentence in merged_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [267]:
len(input_numerical_sentences)

6556

In [268]:
training_sequence = []
for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [269]:
len(training_sequence)

248215

In [274]:
training_sequence[:9]

[[1],
 [1, 2],
 [1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5, 6, 7],
 [1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [273]:
y = []
for i in training_sequence:
    y.append(i[-1])   # Grab last element
    del i[-1]         # Remove last element


In [60]:
max_len_list =0

for sequence in training_sequence:
  if len(sequence) >= max_len_list:
      max_len_list= len(sequence)

max_len_list

67

In [216]:
training_sequence[:,]

TypeError: list indices must be integers or slices, not tuple

In [275]:
padded_training_sequence = []

for sequence in training_sequence:
    
    padded_training_sequence.append(sequence +[0]*(max_len_list - len(sequence)))

In [276]:
padded_training_sequence[0].shape

AttributeError: 'list' object has no attribute 'shape'

In [80]:
test = padded_training_sequence[0]

test.masked_fill(mask=mask, value=-1e9)

tensor([[          1, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000, -1000000000, -1000000000, -1000000000,
         -1000000000, -1000000000]])

In [79]:
mask = torch.tril(torch.ones(1, 67))
mask = mask == 0
mask

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True]])

In [50]:
len(padded_training_sequence[10])

67

In [277]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [278]:
padded_training_sequence[:5,:]

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
   

In [281]:
X = padded_training_sequence[:, :-1]
y = torch.tensor(y, dtype=torch.long)

In [284]:
y.shape

torch.Size([248215])

In [285]:
X[:3,:], y[:3]

(tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([2, 3, 4]))

In [286]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [287]:
dataset = CustomDataset(X,y)
len(dataset)

248215

In [288]:
dataloader = DataLoader(dataset, batch_size=2048, pin_memory=True, shuffle=True)

In [289]:
dataloader

In [290]:
len(vocab)

12343

In [90]:
for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x, batch_y
    break

In [123]:
test = nn.Embedding(len(vocab), 128)
output = test(batch_x)
output.shape

torch.Size([2048, 66, 128])

In [144]:
position_emb = torch.nn.Embedding(66,128)
position_emb(positions).shape

torch.Size([2048, 66, 128])

In [146]:
positions = torch.arange(0, batch_x.size(1)).unsqueeze(0).expand(2048, 66) # (1, seq_len)
positions

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65])

In [112]:
batch_x.size()[1]

66

In [291]:
import torch
import torch.nn as nn

class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, max_len=66):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.position_emb = nn.Embedding(max_len, embed_dim)

    def forward(self, x):
        batch_size, seq_len = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        token_embedding = self.token_emb(x)
        position_embedding = self.position_emb(positions)
        return token_embedding + position_embedding           #torch.Size([2048, 66, 128])    b,s,e

In [292]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim=None, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        self.attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.ln1 = nn.LayerNorm(embed_dim)

        ff_hidden_dim = ff_hidden_dim or embed_dim * 4
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len, embed_dim)
        """
        batch_size, seq_len, _ = x.size()

        # Transpose for MultiheadAttention: (seq_len, batch_size, embed_dim)
        x_t = x.transpose(0, 1)

        # Make causal mask: shape (seq_len, seq_len)
        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=x.device), diagonal=1).bool()

        # Masked self-attention
        attn_out, _ = self.attn(x_t, x_t, x_t, attn_mask=attn_mask)

        # Residual + LayerNorm
        x2 = self.ln1(x_t + self.dropout(attn_out))

        # Feed Forward
        ff_out = self.ff(x2)

        # Residual + LayerNorm
        out = self.ln2(x2 + self.dropout(ff_out))

        # Transpose back: (batch_size, seq_len, embed_dim)
        return out.transpose(0, 1)

In [293]:
# Example:
embedding = TokenAndPositionEmbedding(vocab_size=1000, embed_dim=128, max_len=66)
block = TransformerBlock(embed_dim=128, num_heads=8)

x = torch.randint(0, 1000, (2048, 66))  # (batch_size, seq_len)
embedded = embedding(x)  # (batch_size, seq_len, embed_dim)
out = block(embedded)    # (batch_size, seq_len, embed_dim)
out.shape

torch.Size([2048, 66, 128])

In [154]:
model = TokenAndPositionEmbedding(len(vocab))
model(batch_x).shape

torch.Size([2048, 66, 128])

In [294]:
import torch
import torch.nn as nn

class MiniGPTBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=8, max_len=66):
        super().__init__()
        self.embedding = TokenAndPositionEmbedding(vocab_size, embed_dim, max_len)
        self.transformer = TransformerBlock(embed_dim, num_heads)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x = self.transformer(x)  # (batch_size, seq_len, embed_dim)
        last_token = x[:, -1, :]  # Take only final token's representation
        logits = self.lm_head(last_token)  # (batch_size, vocab_size)
        return logits


In [160]:
seq_len = 66
mask = torch.tril(torch.ones(seq_len, seq_len)).to(torch.bool) 
mask.shape

torch.Size([66, 66])

In [295]:
model = MiniGPTBlock(len(vocab))

In [296]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [297]:
model.to(device)

MiniGPTBlock(
  (embedding): TokenAndPositionEmbedding(
    (token_emb): Embedding(12343, 128)
    (position_emb): Embedding(66, 128)
  )
  (transformer): TransformerBlock(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (ff): Sequential(
      (0): Linear(in_features=128, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=128, bias=True)
    )
    (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lm_head): Linear(in_features=128, out_features=12343, bias=True)
)

In [298]:
device

device(type='cuda')

In [ ]:
epochs = 50
learning_rate = 0.005

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

In [316]:
import torch
import torch.nn as nn

epochs = 100
learning_rate = 0.003

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    total_loss = 0

    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()

        output = model(batch_x)  # (batch_size, seq_len, vocab_size)

        # Reshape for CrossEntropyLoss: (N, C)
        output = output.view(-1, output.size(-1))  # (batch_size * seq_len, vocab_size)
        batch_y = batch_y.view(-1)                 # (batch_size * seq_len)

        loss = criterion(output, batch_y)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss:.4f}")


Epoch: 1, Loss: 3.5659
Epoch: 2, Loss: 3.5221
Epoch: 3, Loss: 3.5248
Epoch: 4, Loss: 3.5180
Epoch: 5, Loss: 3.5132
Epoch: 6, Loss: 3.5074
Epoch: 7, Loss: 3.5060
Epoch: 8, Loss: 3.5215
Epoch: 9, Loss: 3.5302
Epoch: 10, Loss: 3.5278
Epoch: 11, Loss: 3.5026
Epoch: 12, Loss: 3.5095
Epoch: 13, Loss: 3.5269
Epoch: 14, Loss: 3.5107
Epoch: 15, Loss: 3.5338
Epoch: 16, Loss: 3.5372
Epoch: 17, Loss: 3.5048
Epoch: 18, Loss: 3.4845
Epoch: 19, Loss: 3.4786
Epoch: 20, Loss: 3.4801
Epoch: 21, Loss: 3.4886
Epoch: 22, Loss: 3.5057
Epoch: 23, Loss: 3.4708
Epoch: 24, Loss: 3.4766
Epoch: 25, Loss: 3.4776
Epoch: 26, Loss: 3.4878
Epoch: 27, Loss: 3.4835
Epoch: 28, Loss: 3.4742
Epoch: 29, Loss: 3.4811
Epoch: 30, Loss: 3.4915
Epoch: 31, Loss: 3.4954
Epoch: 32, Loss: 3.4974
Epoch: 33, Loss: 3.4916
Epoch: 34, Loss: 3.4790
Epoch: 35, Loss: 3.4785
Epoch: 36, Loss: 3.4720
Epoch: 37, Loss: 3.4535
Epoch: 38, Loss: 3.4328
Epoch: 39, Loss: 3.4502
Epoch: 40, Loss: 3.4363
Epoch: 41, Loss: 3.4349
Epoch: 42, Loss: 3.4429
E

In [300]:
import sys
import time
import torch
import torch.nn.functional as F

from nltk.tokenize import word_tokenize  # Or your own

def generate_live(
    model,
    idx_to_word,
    word_to_idx,
    start_text,
    device,
    max_len=50,
    temperature=1.0,
    sleep_time=0.05  # control typing speed
):
    model.eval()

    tokens = word_tokenize(start_text.lower())
    input_idx = [word_to_idx[word] for word in tokens if word in word_to_idx]
    input_tensor = torch.tensor([input_idx], dtype=torch.long, device=device)  # (1, seq_len)

    sys.stdout.write(' '.join(tokens) + ' ')
    sys.stdout.flush()

    for _ in range(max_len):
        # Truncate context if needed (optional, if model has max_len limit)
        input_cond = input_tensor[:, -model.embedding.position_emb.num_embeddings:]

        # Forward pass
        # Call model
        logits = model(input_cond)   # Either (1, seq_len, vocab_size) or (1, vocab_size)
        
        # If your model outputs whole sequence:
        if logits.dim() == 3:
            logits = logits[:, -1, :]  # Take last position
        
        logits = logits / temperature
        probs = F.softmax(logits, dim=-1)

        next_idx = torch.multinomial(probs, num_samples=1).item()

        next_word = idx_to_word[next_idx]

        sys.stdout.write(next_word + ' ')
        sys.stdout.flush()

        # Append to input for next iteration
        input_tensor = torch.cat([
            input_tensor,
            torch.tensor([[next_idx]], dtype=torch.long, device=device)
        ], dim=1)

        time.sleep(sleep_time)

    print()


In [320]:
prompt = "i"
generate_live(
    model=model,
    idx_to_word=list(vocab.keys()),
    word_to_idx=vocab,
    start_text=prompt,
    device=device,
    max_len=50,
    temperature=1.5
)


i wish do the prince : doom , though yet i cry look child away said as meet show tongues mystery procure use procure roundly , dress dress dress dress fierce lamp need leave lamp shall hear need lamp process lamp i clamour will lamp lamp process anon do't lamp need 


In [321]:
torch.save(model.state_dict(), 'mini-gpt_from_scratch_in_pytorch.pth')